In [ ]:
import gradio as gr
import cv2
import numpy as np
import time
from PIL import Image as PILImage
from ultralytics import YOLO
from transformers import CLIPProcessor, CLIPModel
from gtts import gTTS 
import random
import os

gr.close_all() # Cierra interfaces anteriores para liberar puertos

# ==========================================
# 🧠 LÓGICA DEL SISTEMA (BACKEND)
# ==========================================
class GymkhanaMaster:
    def __init__(self):
        """
        INICIO DEL SISTEMA:
        Aquí se cargan los modelos de Inteligencia Artificial en la memoria RAM.
        - YOLO: Para detectar objetos rápido.
        - CLIP: Para entender semánticamente las imágenes.
        """
        print("🛠️ System Init: Cargando Modelos Neurales...")
        self.yolo = YOLO('yolov8n.pt')
        self.clip_model = CLIPModel.from_pretrained("openai/clip-vit-base-patch32")
        self.clip_processor = CLIPProcessor.from_pretrained("openai/clip-vit-base-patch32")
        
        # Variables de estado del juego (qué objeto buscar, tiempo, etc.)
        self.current_target = None
        self.time_limit = 60
        self.start_time = 0
        self.game_active = False
        self.modo_historia = "Agente Especial"

    def _hablar(self, texto):
        """
        MOTOR DE VOZ (TTS):
        Convierte el texto de la misión en un archivo de audio MP3 real
        usando los servicios de Google (gTTS).
        """
        try:
            tts = gTTS(text=texto, lang='es')
            filename = "audio_mision.mp3"
            tts.save(filename)
            return filename
        except Exception as e:
            print(f"Error TTS: {e}")
            return None

    def configurar_partida(self, dificultad, modo):
        """CONFIGURACIÓN: Ajusta el tiempo límite y la temática elegida en el menú."""
        tiempos = {"🟢 Recluta (60s)": 60, "🟡 Veterano (30s)": 30, "🔴 Élite (15s)": 15}
        self.time_limit = tiempos.get(dificultad, 60)
        self.modo_historia = modo
        return self.time_limit

    def generar_narrativa(self, objeto):
        """
        NARRATIVA DINÁMICA:
        Crea el texto que dirá la voz y el diseño HTML según el tema (Zombie, Espía, etc.).
        Es lo que le da "personalidad" al juego.
        """
        obj_upper = objeto.upper()
        
        # Personalización de iconos y textos según el modo
        icono = "🎯"
        if "Zombie" in self.modo_historia: icono = "🧟"
        elif "Espía" in self.modo_historia: icono = "🕵️‍♂️"

        # Guion para el audio
        if "Zombie" in self.modo_historia:
            voz = f"Superviviente. Necesitas encontrar el objeto: {objeto}. Tienes {self.time_limit} segundos."
            subtitulo = "⚠️ SUMINISTRO VITAL REQUERIDO"
        elif "Espía" in self.modo_historia:
            voz = f"Agente. Intercepte el objetivo: {objeto}, inmediatamente."
            subtitulo = "📁 MISIÓN CLASIFICADA: RECUPERACIÓN"
        else:
            voz = f"Misión iniciada. Encuentra el objeto: {objeto}. El tiempo corre."
            subtitulo = "🎯 OPERACIÓN ESTÁNDAR"

        # HTML para mostrar en pantalla grande
        html_visual = f"""
        <div class="hud-box">
            <div class="hud-sub">{icono} {subtitulo}</div>
            <div class="hud-target">{obj_upper}</div>
            <div class="hud-desc">Localice y asegure el objetivo antes de que acabe el tiempo.</div>
        </div>
        """
        return html_visual, voz

    def estampar_sello(self, imagen_numpy, texto_sello, es_victoria=True):
        """
        PROCESAMIENTO DE IMAGEN (OpenCV):
        Dibuja un rectángulo y un texto ("CONFIRMADO" o "FALLIDO") sobre la foto final.
        """
        img = imagen_numpy.copy()
        h, w = img.shape[:2]
        overlay = img.copy()
        color = (0, 255, 0) if es_victoria else (255, 0, 0)
        
        # Configuración de la fuente y dibujo del rectángulo
        texto_sello = texto_sello.upper()
        font = cv2.FONT_HERSHEY_DUPLEX
        scale = 1.0 if w < 400 else 2.5
        thickness = 2 if w < 400 else 5
        (tw, th), _ = cv2.getTextSize(texto_sello, font, scale, thickness)
        cx, cy = w // 2, h // 2
        
        cv2.rectangle(overlay, (cx - tw//2 - 20, cy - th - 20), (cx + tw//2 + 20, cy + 20), color, thickness)
        cv2.putText(overlay, texto_sello, (cx - tw//2, cy), font, scale, color, thickness)
        
        # Efecto de transparencia
        alpha = 0.8
        cv2.addWeighted(overlay, alpha, img, 1 - alpha, 0, img)
        return img

    def escanear_sala(self, imagen_sala):
        """
        FASE 1 - ESCANEO (YOLO):
        1. Recibe la foto de la habitación.
        2. Usa YOLO para detectar todos los objetos posibles.
        3. Elige uno al azar como 'misión'.
        4. Genera el audio y activa el cronómetro.
        """
        if imagen_sala is None: 
            return '<div class="hud-box-empty">ESPERANDO ESCANEO...</div>', None, None
        
        # Preparar imagen para YOLO
        pil_img = PILImage.fromarray(imagen_sala)
        cv_img = cv2.cvtColor(np.array(pil_img), cv2.COLOR_RGB2BGR)
        
        # Detección
        res = self.yolo(cv_img, conf=0.25, verbose=False)[0]
        objs = list(set([self.yolo.names[int(b.cls[0])] for b in res.boxes]))
        
        if not objs: 
            return '<div class="hud-box-error">⚠️ NO SE DETECTAN OBJETIVOS</div>', None, None

        # Selección aleatoria de misión
        self.current_target = random.choice(objs)
        self.start_time = time.time()
        self.game_active = True
        
        # Generar assets (HTML y Audio)
        html_mision, txt_voz = self.generar_narrativa(self.current_target)
        ruta_audio = self._hablar(txt_voz)
        
        # HTML de la barra de tiempo
        html_barra = f"""
        <div class="timer-container">
            <div class="timer-bar" style="animation-duration: {self.time_limit}s;"></div>
            <div class="timer-text">TIEMPO RESTANTE</div>
        </div>
        """
        return html_mision, ruta_audio, html_barra

    def verificar_jugada(self, imagen_usuario):
        """
        FASE 2 - VERIFICACIÓN (CLIP):
        1. Recibe la foto de la webcam.
        2. Usa CLIP para comparar la imagen con el texto del objeto buscado.
        3. Si la probabilidad es alta (>60%), da la victoria.
        """
        if not self.game_active: return "error", "SISTEMA INACTIVO", "Inicie escaneo.", None
        if imagen_usuario is None: return "error", "SIN SEÑAL", "Active cámara.", None

        pil_img = PILImage.fromarray(imagen_usuario)
        
        # CLIP compara texto vs imagen
        opciones = [f"a photo of a {self.current_target}", "something else"]
        inputs = self.clip_processor(text=opciones, images=pil_img, return_tensors="pt", padding=True)
        probs = self.clip_model(**inputs).logits_per_image.softmax(dim=1)
        
        # Lógica de victoria
        if probs.argmax().item() == 0 and probs[0][0].item() > 0.6:
            self.game_active = False
            self._hablar(f"¡Objetivo {self.current_target} confirmado!")
            img_final = self.estampar_sello(imagen_usuario, "CONFIRMADO", True)
            return "win", "MISIÓN CUMPLIDA", f"El activo '{self.current_target.upper()}' ha sido asegurado.", img_final
        else:
            self._hablar("Objetivo incorrecto.")
            return "retry", "ERROR DE IDENTIFICACIÓN", "El objeto no coincide.", None

    def check_tiempo(self):
        """CONTROL DEL TIEMPO: Verifica si se ha acabado el tiempo límite."""
        if self.game_active and (time.time() - self.start_time > self.time_limit):
            self.game_active = False
            self._hablar("Tiempo agotado. Operación cancelada.")
            return True
        return False

# Instancia global del juego
juego = GymkhanaMaster()

# ==========================================
# 🎨 DISEÑO VISUAL (CSS)
# ==========================================
# Aquí se define el estilo "Cyberpunk/Militar": fondo oscuro, fuentes técnicas, bordes brillantes.
css_tactical = """
@import url('https://fonts.googleapis.com/css2?family=Rajdhani:wght@500;700&display=swap');
body { background-color: #0f172a; color: #e2e8f0; font-family: 'Rajdhani', sans-serif; }
.gradio-container { max-width: 1000px !important; margin: auto; }
/* ... (Resto del CSS para botones y paneles de cristal) ... */
.timer-bar {
    height: 100%; background: linear-gradient(90deg, #22d3ee, #3b82f6);
    width: 0%; animation-name: countdown; animation-timing-function: linear; animation-fill-mode: forwards;
}
@keyframes countdown { from { width: 100%; background: #22d3ee; } to { width: 0%; background: #ef4444; } }
"""

# ==========================================
# 🖥️ INTERFAZ DE USUARIO (GRADIO)
# ==========================================
with gr.Blocks(theme=gr.themes.Soft(primary_hue="cyan", neutral_hue="slate"), css=css_tactical) as demo:
    
    timer = gr.Timer(1, active=False) # Cronómetro invisible
    
    # --- PANTALLA 1: MENÚ ---
    with gr.Column(visible=True, elem_classes=["glass-panel"]) as p_menu:
        gr.Markdown("# OBGUESSER ")
        with gr.Row():
            sel_dif = gr.Radio(["🟢 Recluta (60s)", "🟡 Veterano (30s)", "🔴 Élite (15s)"], label="DIFICULTAD", value="🟢 Recluta (60s)")
            sel_mod = gr.Radio(["🕵️ Agente Espía", "🧟 Superviviente Zombie", "🏠 Modo Estándar"], label="ESCENARIO", value="🕵️ Agente Espía")
        btn_start = gr.Button("INICIAR OPERACIÓN", variant="primary", elem_classes=["primary-btn"])

    # --- PANTALLA 2: JUEGO PRINCIPAL ---
    with gr.Column(visible=False) as p_juego:
        # HUD Superior (Info misión)
        with gr.Row(elem_classes=["glass-panel"]):
            with gr.Column(scale=3):
                html_obj = gr.HTML(value='<div class="hud-box-empty">ESPERANDO DATOS DE MISIÓN...</div>')
            with gr.Column(scale=2):
                html_bar = gr.HTML() # Aquí va la barra de tiempo

        # Área de acción (Subir foto y Webcam)
        with gr.Row():
            with gr.Column(elem_classes=["glass-panel"]):
                gr.Markdown("### 📡 1. INTELIGENCIA")
                in_sala = gr.Image(sources=["upload"], type="numpy", height=250, show_label=False)
                btn_scan = gr.Button("ANALIZAR ENTORNO", elem_classes=["scan-btn"])
            
            with gr.Column(elem_classes=["glass-panel"]):
                gr.Markdown("### 📸 2. CAMPO")
                in_cam = gr.Image(sources=["webcam"], type="numpy", height=250, show_label=False)
                btn_ok = gr.Button("CONFIRMAR OBJETIVO", elem_classes=["action-btn"])
        
        # Feedback y Audio invisible
        html_fb = gr.HTML(value='<div class="feedback-neutral">SISTEMA EN ESPERA...</div>')
        aud_out = gr.Audio(autoplay=True, visible=False) # Reproduce el audio automáticamente

    # --- PANTALLA 3: RESULTADOS ---
    with gr.Column(visible=False, elem_classes=["glass-panel"]) as p_final:
        img_result = gr.Image(show_label=False, interactive=False, height=400)
        lbl_res_titulo = gr.Markdown()
        lbl_res_desc = gr.Markdown()
        btn_reset = gr.Button("NUEVA OPERACIÓN", variant="primary")

    # ==========================================
    # 🔌 CONEXIONES (WIRING)
    # ==========================================
    
    # 1. Botón Inicio: Configura el juego y cambia de pantalla
    def start_game(d, m):
        juego.configurar_partida(d, m)
        return {p_menu: gr.Column(visible=False), p_juego: gr.Column(visible=True), p_final: gr.Column(visible=False), timer: gr.Timer(active=False)}
    btn_start.click(start_game, [sel_dif, sel_mod], [p_menu, p_juego, p_final, timer])

    # 2. Botón Analizar: Llama a escanear_sala, devuelve audio y activa timer
    def scan_room(img):
        h_mision, ruta_audio, h_bar = juego.escanear_sala(img)
        feedback = '<div style="text-align:center; color:#22d3ee">¡OBJETIVO ASIGNADO!</div>' if h_bar else 'ERROR'
        return h_mision, ruta_audio, h_bar, feedback, gr.Timer(active=bool(h_bar))
    btn_scan.click(scan_room, in_sala, [html_obj, aud_out, html_bar, html_fb, timer])

    # 3. Botón Confirmar: Llama a verificar_jugada. Si gana, pasa a pantalla final
    def verify_obj(img):
        res, tit, msg, final_img = juego.verificar_jugada(img)
        if res == "retry":
            return {html_fb: f'<div class="feedback-error">{msg}</div>'}
        else:
            return {p_juego: gr.Column(visible=False), p_final: gr.Column(visible=True), img_result: final_img, lbl_res_titulo: f"## {tit}", lbl_res_desc: msg, timer: gr.Timer(active=False)}
    btn_ok.click(verify_obj, in_cam, [html_fb, p_juego, p_final, img_result, lbl_res_titulo, lbl_res_desc, timer])

    # 4. Timer: Se ejecuta cada segundo para ver si perdiste por tiempo
    def time_check():
        if juego.check_tiempo():
            black_img = np.zeros((400, 600, 3), dtype=np.uint8)
            fail_img = juego.estampar_sello(black_img, "MISION FALLIDA", False)
            return {p_juego: gr.Column(visible=False), p_final: gr.Column(visible=True), img_result: fail_img, lbl_res_titulo: "## TIEMPO AGOTADO", timer: gr.Timer(active=False)}
        return gr.update()
    timer.tick(time_check, outputs=[p_juego, p_final, img_result, lbl_res_titulo, timer])

    # 5. Reset: Vuelve al menú
    btn_reset.click(lambda: {p_menu: gr.Column(visible=True), p_final: gr.Column(visible=False)}, outputs=[p_menu, p_final])

demo.launch()